In [2]:
#written by Laura Z. Fang, Data Science Civic Digital Fellow for the Center of Enterprise Dissemination
#last edited: Aug. 13, 2021

In [1]:
import requests
import urllib.parse as ulp
import mysql.connector

In [2]:
#connect to JQ Admin so we can create&modify databases on JQ Admin
mydb = mysql.connector.connect(
host="web5.ces.census.gov",
user=##,
password=##,
database="pmt"
)
mycursor = mydb.cursor()

#change the encoding of JQ Admin databases so it matches the way the API information is encoded
sql = "SET collation_connection = 'utf8_general_ci'"
mycursor.execute(sql)
mydb.commit()
sql1 = "ALTER TABLE api_publications CONVERT TO CHARACTER SET utf8 COLLATE utf8_general_ci"
mycursor.execute(sql1)
mydb.commit()

In [3]:
# '''populate api_db table'''
# mydb = mysql.connector.connect(
# host="web5.ces.census.gov",
# user="laura",
# password="changeme",
# database="pmt"
# )

# mycursor = mydb.cursor()

# sql = "INSERT INTO api_db (name, url) VALUES (%s, %s)"
# val = ("Base", "https://api.base-search.net/")
# mycursor.execute(sql, val)

# sql = "INSERT INTO api_db (name, url) VALUES (%s, %s)"
# val = ("Core", "https://core.ac.uk/services/api/")
# mycursor.execute(sql, val)

In [4]:
'''if erase all database entries and want to repopulate from scratch, re-run this code block so that 
auto-increment (pub_id values) start back at 1. Otherwise, the first entry will have pub_id of whatever
the old last entry was + 1'''
# mydb = mysql.connector.connect(
#         host="web5.ces.census.gov",
#         user=##,
#         password=##,
#         database="pmt"
#         )

# mycursor = mydb.cursor()

# test = "ALTER TABLE api_publications AUTO_INCREMENT = 1"
# mycursor.execute(test)
# mydb.commit()
# print("altered auto-increment")

'if erase all database entries and want to repopulate from scratch, re-run this code block so that \nauto-increment (pub_id values) start back at 1. Otherwise, the first entry will have pub_id of whatever\nthe old last entry was + 1'

In [5]:
'''code for getting information from CMS'''

from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

#sample url
#https://www.ces.census.gov/api/iaa/v1.3/project/status/CP?fields=cms_project_id,title&page=0&limit=10
#queries CMS database
def run_query(path, fields, page, limit):
    request = requests.get('https://www.ces.census.gov/api/iaa/v1.3%s?fields=%s&page=%s&limit=%s' % (path,fields,page,limit), headers=headers, proxies={"https" : "148.129.75.18:3128"})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}.".format(request.status_code))

#return project title and names of all the project's datasets based on its project id
def name_and_ds(proj_id): 
    data = {}
    proj = run_query('/project/'+str(proj_id),'cms_project_id,title','0','10') # Execute the project query
    #'/project/status/CP'
    full = proj[0]['title']
    parts = full.split("-")
    #print(parts)
    data['title']= parts[2][1:]
    #if there's a '-' in the title, make sure to include it so return the full title
    if len(parts) > 3:
        for i in range(len(parts)-3):
            data['title']= data.get('title')+'-'+parts[3+i]
    #print(proj['cms_project_id'])
    datasets = run_query('/project/%s/data' % proj[0]['cms_project_id'],'name','0','10') # Execute the dataset query
    sets = []

    for ds in datasets:
        sets.append(ds['name'])

    data['datasets']=sets
    return data


In [6]:
'''get abstract from CMS'''
import base64
from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

#builds query url
def run_abstract_query(path,fields,page,limit):
    request = requests.get('https://www.ces.census.gov/api/iaa/v1.3%s?fields=%s&page=%s&limit=%s' % (path,fields,page,limit), headers=headers, proxies={"https" : "148.129.75.18:3128"})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}.".format(request.status_code))

#returns text of the project description corresponding to the given project id. If change "proposal" to
#"abstract", should return the abstract instead, which is shorter than description usually
def get_abstract(projid):
    results = run_abstract_query('/project/'+str(projid)+'/proposal/text','mime_data','0','10')
    #print(results)
    return results[0]['mime_data'].replace("\n", " ")

In [7]:
'''get cms authors'''
import base64
from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

#return all authors with "researcher" role for a given project id (should correspond to all researchers who aren't Census employees)
def get_authors(pjid):
    projects = run_query('/project/'+str(pjid),'cms_project_id,type_name,title,participants{username,fullname,roles}','0','100') # Execute the query

    auts = []
    
    for proj in projects:
        #print(proj['title'])
        if proj['title'].split("-")[0] == str(pjid) + " ":
            for part in proj['participants']:
                if 'researcher' in part['roles']:
                    auts.append(part['fullname'])
                    
    return auts

In [8]:
class All: 
    # sends publication information to JQ Admin databases
    def sendToDBAll(self, db, proj_id, results, hits):
        #print(proj_id)
        mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user=##,
        password=##,
        database="pmt"
        )

        mycursor = mydb.cursor()

        numPrint = 125   
        if hits is not None:
            #print first 125 search results or all results if total is less than 125
            #125 is the limit for amount of results Base will return at one time. If want more, need
            #to retrieve results from specific pages of results (see Base API documentation)
            if(hits < 125):
                numPrint = hits

            #Base
            if db == 1:
                for i in range(numPrint):
                    title = str(results[1]['response']['docs'][i].get('dctitle'))
                    authors = results[1]['response']['docs'][i].get('dcperson')
                    if results[1]['response']['docs'][i].get('dcdoi') is not None:
                        doi = results[1]['response']['docs'][i].get('dcdoi')[0]
                    else: 
                        doi = "None"
                    if results[1]['response']['docs'][i].get('dcdate') is not None:
                        pub_year = str(results[1]['response']['docs'][i].get('dcdate')[:4])
                        pub_month = str(results[1]['response']['docs'][i].get('dcdate')[5:7])
                        pub_day = str(results[1]['response']['docs'][i].get('dcdate')[8:10])
                        if pub_month is '':
                            pub_month = "None"
                        if pub_day is '':
                            pub_day = "None"
                    else:
                        pub_year = "None"
                        pub_month = "None"
                        pub_day = "None"
                    if results[1]['response']['docs'][i].get('dcidentifier') is not None:
                        full_text_link = results[1]['response']['docs'][i].get('dcidentifier')[0]
                    else:
                        full_text_link = "None"
                    if results[1]['response']['docs'][i].get('dcpublisher') is not None:
                        publisher = results[1]['response']['docs'][i].get('dcpublisher')[0]
                    else:
                        publisher = "None"
                    abstract = str(results[1]['response']['docs'][i].get('dcdescription'))
                    doc_id = str(results[1]['response']['docs'][i].get('dcdocid'))
                    if results[1]['response']['docs'][i].get('dctype') is not None:
                        if len(results[1]['response']['docs'][i].get('dctype')) > 1:
                            final_type = ""
                            for j in range(len(results[1]['response']['docs'][i].get('dctype'))):
                                final_type += " " + results[1]['response']['docs'][i].get('dctype')[j]
                            pub_type = final_type
                        else:
                            pub_type = results[1]['response']['docs'][i].get('dctype')[0]
                    else: 
                        pub_type = "None"

                    #query to grab all existing database entries that already have the same proj_id and doc_id as 
                    #entry you're about to add (duplicates)
                    #backticks around database bc it's a reserved word in SQL so need to clarify we're referring to the variable
                    sql_dup = "SELECT * FROM api_publications WHERE doc_id = %s AND proj_id = %s AND db_id = %s"
                    #print(type(proj_id))
                    #print(type(db))
                    data = (doc_id, proj_id, db) 
                    mycursor.execute(sql_dup, data)
                    duplicates = mycursor.fetchall()


                    #if this doc_id doesn't already exist for this proj_id, add it to database
                    if len(duplicates) == 0:
                        sql = "INSERT INTO api_publications (proj_id, db_id, doi, title, pub_year, pub_month, pub_day, publisher, abstract, pub_type, full_text_link, doc_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                        val = (proj_id, db, doi, title, pub_year, pub_month, pub_day, publisher, abstract, pub_type, full_text_link, doc_id)
                        mycursor.execute(sql, val)
                        pub_id = mycursor.lastrowid
                        print("pub_id: " +str(pub_id))

                        mydb.commit()

                        print(mycursor.rowcount, "record inserted.")

                        #if there are authors, insert all authors
                        if bool(authors):
                            for j in range(len(authors)):
                                sql_part = "INSERT INTO api_participants (pub_id, fullname) VALUES (%s, %s)"
                                val_part = (pub_id, authors[j])
                                mycursor.execute(sql_part, val_part)

                                mydb.commit()

                                print(mycursor.rowcount, "participant record inserted.")

                    #if this publication already exists in database, don't add again
                    else:
                        print("duplicate value, not added")

            #Core
            if db == 2:
                for i in range(numPrint):
                    title = str(results[1]['data'][i]['_source']['title'])
                    if results[1]['data'][i]['_source']['doi'] is not None:
                        doi = results[1]['data'][i]['_source']['doi']
                    else:
                        doi = "None"
                    #work_id = str(results[1]['data'][i]['_id'])
                    author = results[1]['data'][i]['_source']['authors']
                    if str(results[1]['data'][i]['_source']['citations']) != '[]':
                        citations = str(results[1]['data'][i]['_source']['citations'])
                    else:
                        citations = "None"
                    contributors = results[1]['data'][i]['_source']['contributors']
                    if results[1]['data'][i]['_source']['datePublished'] is not None:
                        pub_year = str(results[1]['data'][i]['_source']['datePublished'])[:4]
                        pub_month = str(results[1]['data'][i]['_source']['datePublished'])[5:7]
                        pub_day = str(results[1]['data'][i]['_source']['datePublished'])[8:10]
                        if pub_month is '':
                            pub_month = "None"
                        if pub_day is '':
                            pub_day = "None"
                    else:
                        pub_year = "None"
                        pub_month = "None"
                        pub_day = "None"
                    if str(results[1]['data'][i]['_source']['topics']) != '[]':
                        topics = str(results[1]['data'][i]['_source']['topics'])
                    else:
                        topics = "None"
                    full_text = str(results[1]['data'][i]['_source']['fullText'])
                    full_text_link = str(results[1]['data'][i]['_source']['downloadUrl'])
                    publisher = str(results[1]['data'][i]['_source']['publisher'])
                    abstract = str(results[1]['data'][i]['_source']['description'])
                    doc_id = str(results[1]['data'][i]['_id'])

                    sql_dup = "SELECT * FROM api_publications WHERE doc_id = %s AND proj_id = %s AND db_id = %s"
                    data = (doc_id, proj_id, db)
                    mycursor.execute(sql_dup, data)
                    duplicates = mycursor.fetchall()
                    #print(len(duplicates))
                    #for dup in duplicates:
                        #print(dup)
                    if len(duplicates) == 0:
                        sql = "INSERT INTO api_publications (proj_id, db_id, title, doi, pub_year, pub_month, pub_day, topics, citations, abstract, full_text, full_text_link, publisher, doc_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                        val = (proj_id, db, title, doi, pub_year, pub_month, pub_day, topics, citations, abstract, full_text, full_text_link, publisher, doc_id)

                        mycursor.execute(sql, val)
                        pub_id = mycursor.lastrowid
                        print("pub_id: " +str(pub_id))

                        mydb.commit()

                        print(mycursor.rowcount, "record inserted.")

                        # insert all authors into api_participants
                        if bool(author):
                            for j in range(len(author)):
                                sql_part = "INSERT INTO api_participants (pub_id, fullname) VALUES (%s, %s)"
                                val_part = (pub_id, author[j])

                                mycursor.execute(sql_part, val_part)

                                mydb.commit()

                                print(mycursor.rowcount, "participant record inserted.")

                        # insert all contributors into api_participants (Core has two separate fields-- authors and contributors)
                        if bool(contributors):
                            for j in range(len(contributors)):
                                sql_cont = "INSERT INTO api_participants (pub_id, fullname) VALUES (%s, %s)"
                                val_cont = (pub_id, contributors[j])

                                mycursor.execute(sql_cont, val_cont)

                                mydb.commit()

                                print(mycursor.rowcount, "participant record inserted.")

                    else:
                        print("duplicate value, not added")


            #change all "None" entries to null bc null takes up less space and makes for faster query times
            sql_update_year = "UPDATE api_publications SET pub_year=NULL WHERE pub_year='None'"
            mycursor.execute(sql_update_year)
            mydb.commit()
            sql_update_month = "UPDATE api_publications SET pub_month=NULL WHERE pub_month='None'"
            mycursor.execute(sql_update_month)
            mydb.commit()
            sql_update_day = "UPDATE api_publications SET pub_day=NULL WHERE pub_day='None'"
            mycursor.execute(sql_update_day)
            mydb.commit()
            sql_update_doi = "UPDATE api_publications SET doi=NULL WHERE doi='None'"
            mycursor.execute(sql_update_doi)
            mydb.commit()
            sql_update_title = "UPDATE api_publications SET title=NULL WHERE title='None'"
            mycursor.execute(sql_update_title)
            mydb.commit()
            sql_update_link = "UPDATE api_publications SET full_text_link=NULL WHERE full_text_link='None'"
            mycursor.execute(sql_update_link)
            mydb.commit()
            sql_update_publisher = "UPDATE api_publications SET publisher=NULL WHERE publisher='None'"
            mycursor.execute(sql_update_publisher)
            mydb.commit()
            sql_update_abstract = "UPDATE api_publications SET abstract=NULL WHERE abstract='None'"
            mycursor.execute(sql_update_abstract)
            mydb.commit()
            sql_update_id = "UPDATE api_publications SET doc_id=NULL WHERE doc_id='None'"
            mycursor.execute(sql_update_id)
            mydb.commit()
            sql_update_type = "UPDATE api_publications SET pub_type=NULL WHERE pub_type='None'"
            mycursor.execute(sql_update_type)
            mydb.commit()
            sql_update_text = "UPDATE api_core SET full_text=NULL WHERE full_text='None'"
            mycursor.execute(sql_update_text)
            mydb.commit()
            sql_update_topics = "UPDATE api_core SET topics=NULL WHERE topics='None'"
            mycursor.execute(sql_update_topics)
            mydb.commit()
            sql_update_citations = "UPDATE api_core SET citations=NULL WHERE citations='None'"
            mycursor.execute(sql_update_citations)
            mydb.commit()

            print("changed None values to null in doi, title, year, month, pub_type, topics, citations, full text, full text link, publisher, abstract, doc id")

        else:
            print('No hits for this query')

#search Base API
class BaseSearch(All):
            
    def search(self, _path, proj_id, _query):
        header = ["Accept: text/xml"]
        url = "https://api.base-search.net/cgi-bin/BaseHttpSearchInterface.fcgi"
        link = url+_path+"&query="+_query+"&hits=125&format=json"
        request = requests.get(link, proxies={'https':'##'})      
        
        if (request.status_code == 200):
            data = [proj_id, request.json()]
            return data
    
    def searchAll(self, proj_id, _query):
        return self.search("?func=PerformSearch", proj_id, _query)
    
#search Core API
class CoreSearch(All):
        
    def search(self, _path, proj_id, _query):
        header = ["Accept: application/json", "apiKey=##"]
        url = "https://core.ac.uk:443/api-v2"
        #print(url+_path+ulp.quote(_query)+"?page=1&pageSize=20&"+header[1])
        request = requests.get(url+_path+_query+"?page=1&pageSize=125&"+header[1], proxies={'https':'##'})
        
        if (request.status_code == 200):
            data = [proj_id, request.json()]
            return data
    
    def searchAll(self, proj_id, _query):
        return self.search("/search/", proj_id, _query)

In [9]:
'''given an author/researcher name, makes various formats of it to use as search queries for the APIs'''
def aut_queries(author):
    #final array to hold all the author queries we'll be running, without repeats
    aut_queries = []
    #max possible six versions of author's name, with repeats
    authors = []

    part = author.partition(' ')
    first = part[0]
    first_initial = first[0:1] + '.'
    if part[2].partition(' ')[2] is '':
        last = part[2].partition(' ')[0]
    else: 
        middle = part[2].partition(' ')[0]
        middle_initial = middle[0:1] + '.'
        last = part[2].partition(' ')[2]
        aut5 = first_initial + ' ' + middle_initial + ' ' + last
        authors.append(aut5)
    #original author name provided
    aut1 = author
    #author name without middle name or initial
    aut2 = first + ' ' + last
    #if first, middle, or last name is just an initial, add a "." to the end
    #if first, middle, or last name is already an initial with a '.', get rid of the '.'
    #since queries are exact, sometimes 'J. James' returns different results from 'J James'
    if len(first) == 1:
        first = first+"."
    elif '.' in first:
        first = first.partition('.')[0]
    if part[2].partition(' ')[2] is not '':
        if len(middle) == 1:
            middle = middle + "."
        elif '.' in middle:
            middle = middle.partition('.')[0]
    #queries with '.' added in where applicable
    aut3 = first + " " + last
    authors.append(aut1)
    authors.append(aut2)
    authors.append(aut3)
    if part[2].partition(' ')[2] is not '':
        aut4 = first + " " + middle + " " + last
        authors.append(aut4)
    aut6 = first_initial + " " + last
    authors.append(aut6)
    #get rid of duplicate queries between the 4 different versions
    for i in range(len(authors)):
        if authors[i] not in aut_queries:
            aut_queries.append(authors[i])
    #print(queries)
    return aut_queries

In [10]:
'''runs all search queries for a given project id in the base API (project title, variations of lead author's name)'''
def auto_search(db, pjid):
    #array to hold all the query phrases/words we'll be running
    queries = []
    queries.append(name_and_ds(pjid).get('title'))
    # all names listed as researcher in CMS project
    cms_auts = get_authors(pjid)
    for i in range(len(cms_auts)):
        # all versions of each researcher name to search
        q = aut_queries(cms_auts[i])
        for j in range(len(q)):
            queries.append(q[j])
    print(queries)
   
    if db == 1:
        api = BaseSearch()
        for i in range(len(queries)):
            results = api.searchAll(pjid, '"'+queries[i]+'"')
            print("'"+queries[i]+"'")
            #number of hits the query returned
            hits = results[1]['response']['numFound']
            print("Total hits: " + str(hits))
            #print("proj_id: " +str(results[0]))
            #proj_id = str(results[0])
            #print(results)
            #add hits to JQ Admin database
            api.sendToDBAll(db, pjid, results, hits)
    if db == 2:
        api = CoreSearch()
        for i in range(len(queries)):
            results = api.searchAll(pjid, '"'+queries[i]+'"')
            print("'"+queries[i]+"'")
            print("Total hits: " + str(results[1]['totalHits']))
            #print("proj_id: " +str(results[0]))
            hits = results[1]['totalHits']
            #proj_id = str(results[0])
            api.sendToDBAll(db, pjid, results, hits)


In [11]:
#imports for NLP stuff
import numpy as np
import nltk
#nltk.set_proxy('http://proxy.tco.##')
#nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import FreqDist
from nltk import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
# of points per pub_id (a unique ID given to each publication) for each pjid
res_pubs = {} 
# of points assigned to publications from each of the 5 queries (publication year, authors, NLP title, NLP abstract, datasets). PJID is key, value is array with 5 entries
matches = {} 
#parameters used to assign points to publications, for the user's reference if they want to see the NLP search words used, or any of the other metadata
params = {}

In [13]:
'''comprehensive method that runs everything to find most likely hits and give scores to publications, including all NLP stuff'''
def search_NLPAll(pjid, year):
    
    #connect to JQ Admin
    mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user="##",
        password="##",
        database="pmt"
        )

    mycursor = mydb.cursor()

    '''NLP'''
    title = name_and_ds(pjid)['title']
    description = title+' '+get_abstract(pjid)

    #tokenizing words and getting rid of stop words
    descrip = word_tokenize(description)

    stop_words = set(stopwords.words("english"))

    #list with each description word as an entry, and stopwords removed
    filtered_descrip = []
    filtered_list = [
        word for word in descrip if word.casefold() not in stop_words]
    #print(filtered_list)

    #giving each word a part-of-speech tag
    descrip_pos = nltk.pos_tag(descrip)

    #chunking-- looking for phrases with a specific format
    #find noun phrase that starts w/any # adjs, and ends w/any # nouns
    grammar = "NP: {<JJ>*<NN>*}"
    chunk_parser = nltk.RegexpParser(grammar)
    #make a tree out of the chunks
    tree = chunk_parser.parse(descrip_pos)
    #temp array to store all chunks/phrases of a set size
    phrase = []
    # stores all the phrases, key indexed by the # of words in the phrase
    chunks = {}
    for s in tree.subtrees(filter=lambda tree: tree.label() == 'NP'):
        #print([x[0] for x in s.leaves()])
        for x in s.leaves():
            phrase.append(x[0])
        #print(phrase)
        size = len(phrase)
        full = ""
        #format each noun phrase so that each word is separated by a space
        for i in range(size):
            if full=="":
                full = phrase[i]
            else:
                full = full+" "+phrase[i]
        #add phrase to the dictionary under the key value that equals the size of the phrase
        if chunks.get(str(size)) is not None:
            chunks.get(str(size)).append(full)
        else:
            chunks[str(size)] = [full]
        phrase = []

    #frequency distribution
    #stores up to the top 20 most frequent words in the description
    from nltk import FreqDist #without this import statement, it thinks FreqDist is a variable, even though this package was imported in a previous code block
    freq_words = []
    frequency_distribution = FreqDist(filtered_list)
    #print(frequency_distribution.most_common(20))
    for i in range(20):
        word = frequency_distribution.most_common(20)[i][0]
        #if a punctuation mark or article is in the top 20, don't add it to the list
        if len(word) == 1:
            continue
        else:
            freq_words.append(word)

    # #tried using collocations, but the documents aren't big enough
    #print(collections.counter())

    #collocation- sequence of words that shows up often
    # print("collocations")
    # print(nltk.Text(description).collocation_list())

    #ngrams, groups of n words
    from nltk import ngrams, FreqDist
    all_ngrams = {}
    #dictionary where keys are number of words in phrase, and values are the phrases
    all_counts = {}
    for size in 2, 3, 4, 5:
        all_counts[size] = FreqDist(ngrams(filtered_list, size))
    #print(all_counts[2].most_common(5))
    third=''
    fourth=''
    fifth=''
    #get five most ngrams for n= 2,3,4,5
    for i in range(4):
        for j in range(5):
            #first and second words in ngram
            first = all_counts[i+2].most_common(5)[j][0][0]
            second = all_counts[i+2].most_common(5)[j][0][1]
            if i+2 == 2:
                #if at least one of the words is only one character (most likely a punctuation mark), don't add ngram to dictionary
                if len(first) == 1 or len(second) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams[str(i+2)] = all_ngrams.get(str(i+2)).append(first + " " + second)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second]

    #final array of all the keywords we'll be searching for in the publication database (api_base)
    keywords = []
    #add all ngrams where n=2 (once n gets to be >2 the ngrams aren't as coherent because we're missing articles, conjunctions, etc)
    if all_ngrams.get("2") is not None:
        keywords += all_ngrams.get("2")
    #add noun phrases of lengths 2 and 3
    if chunks.get("2") is not None:
        keywords += chunks.get("2")
    if chunks.get("3") is not None:
        keywords += chunks.get("3")
    #add most freq words
    if len(freq_words) > 0:
        keywords += freq_words

    #gets rid of duplicate words or phrases
    kw_set = set(keywords) #caps sensitive-- same words but dif capitalization don't get removed
    keywords = list(kw_set)
    print("done NLP") #done with word/phrase-grabbing part of NLP

    '''end NLP part 1'''


    '''inputs'''

    # keys are the names of all the authors/researchers for this project, values are the variations on their names to search
    authors = {}

    auts = get_authors(pjid) #array of all those listed as researcher in CMS
    numAuts = len(auts)
    title = name_and_ds(pjid)['title']
    # for each author, split into multiple search queries, as we did before
    for i in range(numAuts):
        authors[i] = aut_queries(auts[i])

    ds = name_and_ds(pjid)['datasets'] #array

    #contains search queries for this specific proj_id
    params_inner = {}
    #maximum possible score a publication can have
    max_score = 1 + numAuts #pub_year point plus number of authors

    params_inner['year'] = year
    params_inner['authors'] = auts
    params_inner['datasets'] = ds
    params_inner['keywords'] = keywords
    params[pjid] = params_inner

    #dictionary of how many hits each query gets
    data = {}
    #keys are pub_id, values are the score
    rows = {}

    #add all publications with a feasbile publication year (or null pub year value) to dictionary
    '''pub_year'''
    q1 = "SELECT * FROM api_publications WHERE proj_id= %s AND (pub_year >= %s OR pub_year is NULL)"
    d1 = (pjid, year)
    mycursor.execute(q1, d1)
    r1 = mycursor.fetchall()
    data['pub year'] = len(r1)
    for i in range(len(r1)):
        rows[r1[i][0]] = 1

    '''authors'''
    aut_hits = 0

    def refineAut(pjid, authors):
        q2 = "SELECT A.pub_id, P.proj_id, A.fullname FROM api_participants A LEFT JOIN api_publications P on A.pub_id = P.pub_id WHERE P.proj_id= %s AND (A.fullname LIKE %s"
        d2 = (pjid, str(('%' + authors[0] + '%')))
        l2 = list(d2)
        numData = len(authors)
        if numData == 1:
            q2 += ")"
        else:
            for j in range(numData-1):
                q2 += " OR A.fullname LIKE %s"
                if j == numData-2:
                    q2 += ")"
                l2.append(str(('%' + authors[j+1] + '%')))
        d2 = tuple(l2)
        mycursor.execute(q2, d2)
        r2 = mycursor.fetchall()

        #get one point for each version of an author's name present (should just be max 1 point possible)
        for i in range(len(r2)):
            if rows.get(r2[i][0]) is not None:
                rows[r2[i][0]] = rows.get(r2[i][0])+1
        return len(r2)
    #print(rows)

    #calculates total points received from all author names the publication has
    for i in range(numAuts):
        aut_hits += refineAut(pjid, authors.get(i))
        #print(authors.get(i))
        #print(aut_hits)

    data['author'] = aut_hits
    
    #entries that don't have any of the correct authors will have scores ending in 0.5
    for key in rows.keys():
        if rows.get(key) == 1:
            rows[key] = 0.5

    #same functions as their sister arrays without 'nlp_' in the name, but just for the NLP searches (for data analysis purposes, if interested)
    #I made separate data structures specifically for the NLP stuff so you can run analysis on just the NLP if desired,
    #and see how much influence the NLP methods have on scoring
    nlp_matches = {}
    nlp_data = []
    nlp_rows = {}
    nlp_res = {}

    '''queries 3 and 4: one pt each for having at least one word in the query list, 1 additional pt for each matching word'''
    '''NLP keywords (taken from title and abstract) in description'''
    q3 = "SELECT * FROM api_publications WHERE proj_id= %s AND (abstract LIKE %s"
    d3 = (pjid, str('%' + keywords[0] + '%'))
    l3 = list(d3)
    numData = len(keywords)
    #add all nlp keywords to search query
    if numData == 1:
        q3 += ")"
    else:
        for i in range(numData-1):
            q3 += " OR abstract LIKE %s"
            if i == numData-2:
                q3 += ")"
            # 'like' keyword and % means will get a point as long as keyword is in text, with any characters before and after it 
            l3.append(str('%' + keywords[i+1] + '%'))
    d3 = tuple(l3)
    mycursor.execute(q3, d3)
    r3 = mycursor.fetchall()
    nlp_data.append(len(r3)) #amount of hits that have at least one matching word
    #for each publication that has at least one matching word, give a point for each matching word
    for i in range(len(r3)):
        if nlp_rows.get(r3[i][0]) is not None:
            nlp_rows[r3[i][0]] = nlp_rows.get(r3[i][0])+1
        else:
            nlp_rows[r3[i][0]] = 1

    #exact same as above, but with matching words in the publication title instead of the abstract
    '''NLP keywords in title'''
    q4 = "SELECT * FROM api_publications WHERE proj_id= %s AND (title LIKE %s"
    d4 = (pjid, str('%' + keywords[0] + '%'))
    l4 = list(d4)
    numData = len(keywords)
    if numData == 1:
        q4 += ")"
    else:
        for i in range(numData-1):
            q4 += " OR title LIKE %s"
            if i == numData-2:
                q4 += ")"
            l4.append(str('%' + keywords[i+1] + '%'))
    d4 = tuple(l4)
    mycursor.execute(q4, d4)
    r4 = mycursor.fetchall()
    nlp_data.append(len(r4)) #amount of hits with at least one matching word
    for i in range(len(r4)):
        if nlp_rows.get(r4[i][0]) is not None:
            nlp_rows[r4[i][0]] = nlp_rows.get(r4[i][0])+1
        else:
            nlp_rows[r4[i][0]] = 1
            
    max_score += 2 + 2*numData #max # of points a publication can get from queries 3 and 4

    #grab and store titles and descriptions of all publications that had at least one match in title or descrip
    descrips = {}
    for key in nlp_rows.keys():
        #print(type(key))
        q = "SELECT title FROM api_publications WHERE pub_id = " +str(key)
        mycursor.execute(q)
        r = mycursor.fetchall()
        descrips[key] = [r[0][0]]
        qq = "SELECT abstract FROM api_publications WHERE pub_id = " +str(key)
        mycursor.execute(qq)
        rr = mycursor.fetchall()
        descrips.get(key).append(rr[0][0])

    #print(descrips)
    #print(nlp_rows)

    #total points assigned to all publications from q4
    title_hits = 0
    #total points assigned to all publications from q3
    descrip_hits = 0

    #print(keywords[0])
    #if descrips.keys() is not None:
    
    #giving one additional point to the publication for each word it has
    for i in range(len(keywords)):
        for key in descrips.keys():
            if descrips[key][0] is not None:
                if keywords[i] in descrips[key][0]:
                    if nlp_rows[key] is not None:
                        nlp_rows[key] = nlp_rows.get(key)+1
                        title_hits += 1
            if descrips[key][1] is not None:
                if keywords[i] in descrips[key][1]:
                    if nlp_rows[key] is not None:
                        nlp_rows[key] = nlp_rows.get(key)+1
                        descrip_hits += 1
    
    data['NLP title'] = title_hits
    data['NLP description'] = descrip_hits
     
    '''cosine similarity'''
    #TF-IDF: term frequency, inverse document frequency
    #words that appear more freq across the documents (publication descriptions) get smaller weights
    #words that appear in all get removed entirely. This gets rid of fillers, etc.
    #cosine measures are fracs from 0 to 1 that measure how similar docs are
    #don't add up to 1 but 1 is highest
    
    #stores all text blocks we'll be comparing across to find cosine similarities
    descrips_sim = []
    #keys are index in descrips_sim array, values are the pub_ids
    indices = {}
    counter_sim = 1
    
    descrips_sim.append(description)
    #print(description)
    indices[0] = -1 #index 0 refers to actual description
    #to be used for cosine similarity
    for key in descrips.keys():
        #if publication has a title and description/abstract then append those concatenated together
        if descrips[key][0] is not None and descrips[key][1] is not None:
            descrips_sim.append(descrips[key][0]+" " + descrips[key][1])
            indices[counter_sim] = key
            counter_sim +=1
        elif descrips[key][0] is not None:
            descrips_sim.append(descrips[key][0])
            indices[counter_sim] = key
            counter_sim +=1
        elif descrips[key][1] is not None:
            descrips_sim.append(descrips[key][1])
            indices[counter_sim] = key
            counter_sim +=1 
    
    #create a 2D array that has the cosine similarity scores of each text block as its entries
    #diagonal is all 1s, all entries in row 0 are cosine similarity scores of text blocks with description 0, etc.
    vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
    tfidf = vect.fit_transform(descrips_sim)                                                                                                                                                                                                                       
    pairwise_similarity = tfidf * tfidf.T 

    arr = pairwise_similarity.toarray()     
    np.fill_diagonal(arr, np.nan)                                                                                                                                                                                                                            
    
    #description from CMS-- we want to know how similar all other text blocks are to this one
    input_doc = descrips_sim[0]                                                                                                                                                                                                  
    input_idx = descrips_sim.index(input_doc) #should always be 0                                                                                                                                                                                                                    
    #input_idx                                                                                                                                                                                                                                                

    #a list of lists-- each inner list has two values: the pub_id and its cosine similarity score
    #sorted so first entry has highest cosine similarity
    sorted_sims_final = []

    for i in range(len(arr[input_idx])-1):
        result_idx = np.nanargmax(arr[input_idx])
        best = indices.get(result_idx)
        sorted_sims_final.append([best,arr[input_idx][result_idx]])
        arr[input_idx][result_idx] = -1
       
    #print(sorted_sims_final)
    
    highest = sorted_sims_final[0][1] #highest cosine similarity score value
    for i in range(len(sorted_sims_final)):
        pubID = sorted_sims_final[i][0]
        cosine = sorted_sims_final[i][1]
        #if cosine similarity raw score is higher than 0.5, get a point
        if cosine > 0.5:
            nlp_rows[pubID] = nlp_rows.get(pubID)+1
        #publication with the highest cosine similarity automatically gets 2 points
        if i == 0:
            nlp_rows[pubID] = nlp_rows.get(pubID)+2
        else:
            #if cosine similarity score is at least 90% of the highest cosine similarity among these texts, get 2 points
            if cosine/highest >= 0.9:
                nlp_rows[pubID] = nlp_rows.get(pubID)+2
            elif cosine/highest >= 0.75:
                nlp_rows[pubID] = nlp_rows.get(pubID)+1
            #elif cosine/highest >= 0.5:
               # nlp_rows[pubID] = nlp_rows.get(pubID)+1
        
    max_score += 3
    
    #print(nlp_rows)
    nlp_matches[pjid] = nlp_data
    #print(nlp_matches)
    nlp_res[pjid] = nlp_rows

    #publication gets a point for every dataset its abstract mentions
    '''datasets in description'''
    numData = len(ds)
    dataset_hits = 0
    for i in range(numData):
        q5 = "SELECT * FROM api_publications WHERE proj_id= %s AND abstract LIKE %s"
        d5 = (pjid, ds[i])
        mycursor.execute(q5, d5)
        r5 = mycursor.fetchall()
        dataset_hits += len(r5)
        for i in range(len(r5)):
            if rows.get(r5[i][0]) is not None:
                rows[r5[i][0]] = rows.get(r5[i][0])+1
    
    data['dataset'] = dataset_hits
    max_score += numData

    #same as rows, but sorted so that pub_id's with highest scores are listed first
    res_sorted = {}

    #sort results so that pub_ids with most points are first
    for key in rows.keys():
        if nlp_rows.get(key) is not None:
            rows[key] = rows.get(key)+nlp_rows.get(key)
    #print(rows)

    for w in sorted(rows, key = rows.get, \
                    reverse = True):
        res_sorted[w] = rows[w]
        #print(w, rows[w])
        
    #put score values in database
    for key in res_sorted.keys():   
        qu = "UPDATE api_publications SET score= %s WHERE pub_id = %s"
        du = ((res_sorted.get(key)), key)
        mycursor.execute(qu, du)

    print("max. possible score:")
    print(max_score)
    print("sorted results:")
    
    res_pubs[pjid] = res_sorted
    print(res_pubs[pjid])
    matches[pjid] = data
    print("matches:")
    print(matches)

In [25]:
auto_search(1, 784)
auto_search(2, 784)

['The Effects of Interactions with International Markets on US Firm Performance', 'William R Kerr', 'William Kerr', 'William R. Kerr', 'Jim A Levinsohn', 'Jim Levinsohn', 'Jim A. Levinsohn', 'Jagadeesh M Sivadasan', 'Jagadeesh Sivadasan', 'Jagadeesh M. Sivadasan', 'William Lincoln', 'Andrew McCallum', 'Ryan P Monarch', 'Ryan Monarch', 'Ryan P. Monarch', 'Aaron B Flaaen', 'Aaron Flaaen', 'Aaron B. Flaaen', 'Fariha Kamal', 'Jenny Lin', 'Vanessa I Alviarez', 'Vanessa Alviarez', 'Vanessa I. Alviarez', 'Andrei A Levchenko', 'Andrei Levchenko', 'Andrei A. Levchenko', 'Ihab Saed', 'Lin Ma', 'Rishi Sharma', 'Eric T Chyn', 'Eric Chyn', 'Eric T. Chyn', 'Nitya Pandalai Nayar', 'Nitya Nayar', 'Christoph Boehm']
'The Effects of Interactions with International Markets on US Firm Performance'
Total hits: 0
changed None values to null in doi, title, year, month, pub_type, topics, citations, full text, full text link, publisher, abstract, doc id
'William R Kerr'
Total hits: 226
pub_id: 12476
1 record i

In [32]:
search_NLPAll(263,2002)

done NLP
max. possible score:
123
sorted results:
{5012: 22.5, 638: 20, 5011: 19.5, 639: 17, 4949: 17, 629: 16, 636: 16, 641: 16, 682: 16, 4952: 16, 4964: 16, 5591: 16, 5593: 16, 684: 15, 640: 14, 679: 14, 681: 13, 4960: 13, 4966: 13, 5578: 13, 5486: 12.5, 5488: 12.5, 637: 12, 642: 12, 678: 12, 4858: 12, 4866: 12, 4871: 12, 4910: 12, 4915: 12, 4919: 12, 4927: 12, 4936: 12, 5402: 12, 5515: 12, 633: 11, 4851: 11, 4852: 11, 4874: 11, 4904: 11, 4921: 11, 4925: 11, 4941: 11, 4942: 11, 4956: 11, 4959: 11, 4961: 11, 4965: 11, 4967: 11, 5409: 11, 5410: 11, 5417: 11, 5512: 11, 5560: 11, 5561: 11, 5568: 11, 5571: 11, 657: 10.5, 662: 10.5, 665: 10.5, 698: 10.5, 4969: 10.5, 5010: 10.5, 5496: 10.5, 5503: 10.5, 5563: 10.5, 5606: 10.5, 5611: 10.5, 677: 10, 4617: 10, 4856: 10, 4863: 10, 4873: 10, 4875: 10, 4877: 10, 4930: 10, 4939: 10, 4943: 10, 4946: 10, 4962: 10, 5399: 10, 5416: 10, 5420: 10, 5421: 10, 5520: 10, 5616: 10, 667: 9.5, 669: 9.5, 670: 9.5, 685: 9.5, 686: 9.5, 693: 9.5, 696: 9.5, 4878: 9.

In [21]:
'''comparing results between core and base'''
mydb = mysql.connector.connect(
host="web5.ces.census.gov",
user="##",
password="##",
database="pmt"
)
mycursor = mydb.cursor()

In [1]:
#store info about pub. date, authors, doi, etc. for all core publications
core_info = {}
#same as above but for base publications
base_info = {}
#main dictionary for core, with keys as CMS proj_ids, and values as res_core_scores
#res_core and res_base should be the same as res_pubs, except core only has core publications and similarly for base
res_core = {}
res_core_scores = {}
#main dictionary for base, with keys as CMS proj_ids, and values as res_core_scores
res_base = {}
res_base_scores = {}
prev = 0

#separates publications into two dictionaries-- one for those from Base and one for those from Core
for key in res_pubs.keys():
    for key2 in (res_pubs.get(key)).keys():
        db = "SELECT db_id FROM api_publications WHERE pub_id = " + str(key2)
        mycursor.execute(db)
        dbdb = mycursor.fetchall()
        if dbdb[0][0] == 2:
            res_core_scores[key2] = res_pubs.get(key).get(key2)
        elif dbdb[0][0] == 1:
            res_base_scores[key2] = res_pubs.get(key).get(key2)
    res_core[key] = res_core_scores
    res_base[key] = res_base_scores

NameError: name 'res_pubs' is not defined

In [46]:
#grab and store titles and descriptions of all publications that had at least one match in title or descrip
for key in res_core.keys():
    temp = {}
    for key2 in (res_core.get(key)).keys():
        if prev == 0:
            # get score corresponding to this pub_id (key2) for this proj_id (key)
            prev = (res_core.get(key)).get(key2) #res_core.get...
        curr = (res_core.get(key)).get(key2)
        # only grab info for the pub_id if score is greater than 2 and less than 2 lower than the previous score
        # keys are ordered so we visit higher scoring ones first
        if curr > 2 and curr > prev-3:   
            # grab and store the following info for the pub_id
            q = "SELECT title, doi, publisher, pub_year, pub_month, pub_day, doc_id FROM api_publications WHERE pub_id = " +str(key2)
            mycursor.execute(q)
            qq = mycursor.fetchall()
            temp[key2] = [qq[0][0], qq[0][1], qq[0][2], qq[0][3], qq[0][4], qq[0][5], qq[0][6]]
            a = "SELECT fullname FROM api_participants WHERE pub_id = " +str(key2)
            mycursor.execute(a)
            aa = mycursor.fetchall()
            authors = []
            for i in range(len(aa)):
                authors.append(aa[i][0])
            temp.get(key2).append(authors)
            prev = curr
        else:
            break
    core_info[key] = temp

[('Who Dies? International Trade, Market Structure, and Industrial Restructuring', None, None, None, None, None, '6325526')]
[('Firms in International Trade', '10.2139/ssrn.1015617', "'Elsevier BV'", 2007, 1, 1, '277305840')]
[('Who Dies? International Trade, Market Structure, and Industrial Restructuring', '10.2139/ssrn.272140', None, 2001, None, None, '21510414')]
[('including ©notice, is given to the source. Falling Trade Costs, Heterogeneous Firms, and Industry Dynamics', None, None, 2003, None, None, '103349893')]
[('Measuring the Impact of Trade in Services: Prospects and Challenges', None, None, 2009, None, None, '21693346')]
[('including © notice, is given to the source. Transfer Pricing by U.S.-Based Multinational Firms', None, None, 2006, None, None, '100739309')]
[('Exporting and Productivity: The Importance of Reallocation', None, None, None, None, None, '6464432')]
[('and the (Uneven) Growth of US Manufacturing Plants ∗', None, None, 2003, None, None, '21075048')]
[('Impor

In [48]:
'''get base data'''
#grab and store titles and descriptions of all publications that had at least one match in title or descrip
for key in res_base.keys():
    temp = {}
    for key2 in (res_base.get(key)).keys():
        if prev == 0:
            # get score corresponding to this pub_id (key2) for this proj_id (key)
            prev = (res_base.get(key)).get(key2)
        curr = (res_base.get(key)).get(key2)
        # only grab info for the pub_id if score is greater than 2 and less than 2 lower than the previous score
        # keys are ordered so we visit higher scoring ones first
        if curr > 2 and curr > prev-3:   
            # grab and store the following info for the pub_id
            q = "SELECT title, doi, publisher, pub_year, pub_month, pub_day, doc_id FROM api_publications WHERE pub_id = " +str(key2)
            mycursor.execute(q)
            qq = mycursor.fetchall()
            temp[key2] = [qq[0][0], qq[0][1], qq[0][2], qq[0][3], qq[0][4], qq[0][5], qq[0][6]]
            a = "SELECT fullname FROM api_participants WHERE pub_id = " +str(key2)
            mycursor.execute(a)
            aa = mycursor.fetchall()
            authors = []
            for i in range(len(aa)):
                authors.append(aa[i][0])
            temp.get(key2).append(authors)
            prev = curr
        else:
            break
    base_info[key] = temp

In [54]:
'''comparing results between core and base'''
mydb = mysql.connector.connect(
host="web5.ces.census.gov",
user="##",
password="##",
database="pmt"
)
mycursor = mydb.cursor()

base_core_matches = {}
pjid = 5
for key in base_info.get(pjid).keys():
    # keys are base pub_ids, values are core pub_ids and how many things they have in common
    similar = {}
    for key2 in core_info.get(pjid).keys():
        # same doi
        if base_info.get(pjid).get(key)[1] == core_info.get(pjid).get(key2)[1] and base_info.get(pjid).get(key)[1] is not None:
            similar[key2] = 100
            break
        # same pub day, month, and year
        if base_info.get(pjid).get(key)[3] == core_info.get(pjid).get(key2)[3] and base_info.get(pjid).get(key)[3] is not None:
            similar[key2] = 10
            if base_info.get(pjid).get(key)[4] == core_info.get(pjid).get(key2)[4] and base_info.get(pjid).get(key)[4] is not None:
                similar[key2] = similar.get(key2) + 25
                if base_info.get(pjid).get(key)[5] == core_info.get(pjid).get(key2)[5] and base_info.get(pjid).get(key)[5] is not None:
                    similar[key2] = similar.get(key2) + 50
        # same title
        if base_info.get(pjid).get(key)[0] == core_info.get(pjid).get(key2)[0] and base_info.get(pjid).get(key)[0] is not None:
            #if the key already has a score, add to it. Else, create the key
            if similar.get(key2) is not None:
                similar[key2] = similar.get(key2) + 20
            else:
                similar[key2] = 20
        #same publisher
        if base_info.get(pjid).get(key)[2] == core_info.get(pjid).get(key2)[2] and base_info.get(pjid).get(key)[2] is not None:
            if similar.get(key2) is not None:
                similar[key2] = similar.get(key2) + 10
            else:
                similar[key2] = 10
        # same authors
        if base_info.get(pjid).get(key)[6] is not None: 
            if core_info.get(pjid).get(key) is not None:
                for i in range(len(base_info.get(pjid).get(key)[6])):
                    for j in range(len(core_info.get(pjid).get(key)[6])):
                        if base_info.get(pjid).get(key)[6][i] in core_info.get(pjid).get(key2)[6][j]:
                            if similar.get(key2) is not None:
                                similar[key2] = similar.get(key2) + 15
                            else:
                                similar[key2] = 15

    similar_sorted = {}
    for w in sorted(similar, key = similar.get, \
                    reverse = True):
        similar_sorted[w] = similar[w]
    base_core_matches[key] = similar_sorted
    
#dictionary where key is the pub_id of a base publication and value is another dictionary
#value dictionary has keys that are pub_ids of core publications and values that are scores of 
#how similar the core publication is the base publication (key of the outer dictionary)
print(base_core_matches)

{115: {4238: 20, 429: 20}, 297: {396: 20, 430: 20, 407: 20}, 311: {4238: 10, 367: 10, 497: 10, 4421: 10, 4481: 10, 4234: 10, 4276: 10, 4503: 10, 4616: 10, 4540: 10, 4417: 10, 343: 10, 4353: 10, 4488: 10, 4490: 10, 4492: 10, 4495: 10, 4496: 10, 4499: 10, 4500: 10, 4585: 10, 4593: 10}, 77: {4269: 20}, 104: {}, 112: {321: 20}, 124: {395: 20, 409: 20, 417: 20, 419: 20, 416: 20}, 306: {4238: 20, 429: 20}, 4198: {4238: 20, 429: 20}, 45: {}, 46: {}, 80: {}, 96: {}, 105: {}, 106: {408: 20}, 140: {}, 156: {4238: 30, 429: 20, 367: 10, 497: 10, 4421: 10, 4481: 10, 4234: 10, 4276: 10, 4503: 10, 4616: 10, 4540: 10, 4417: 10, 343: 10, 4353: 10, 4488: 10, 4490: 10, 4492: 10, 4495: 10, 4496: 10, 4499: 10, 4500: 10, 4585: 10, 4593: 10}, 157: {332: 30, 390: 10, 428: 10, 435: 10, 4249: 10, 316: 10, 4434: 10, 350: 10, 4433: 10, 572: 10, 4369: 10, 4461: 10, 4470: 10, 4248: 10, 4250: 10, 4411: 10, 4464: 10, 4507: 10, 4589: 10, 4600: 10, 4315: 10, 4395: 10, 4314: 10, 4316: 10, 4400: 10, 4508: 10}, 163: {4269